<a href="https://colab.research.google.com/github/Yashnikam-10/Task-3-Code-Implemenatation/blob/main/WhisperASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jan 20 06:52:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-5j6in21o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-5j6in21o
  Resolved https://github.com/huggingface/transformers to commit 3f69f415adcbdaedec154ba8eac220ef3276975d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8404406 sha256=cf2c3ea3852409eab25ba76794315bb618c89355e61c26edd8c0cf53b6df88a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-xdky_nh3/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.0.dev0
    Uninstalling transformers-4.38.0.dev0:
      Successfully uninstalled transformers-4.38.0.dev0


In [ ]:
import pandas as pd
from datasets import Dataset

# Load your dataset
df = pd.read_csv("/content/drive/MyDrive/Medical Speech, Transcription, and Intent/overview-of-recordings.csv")
df_sorted = df.sort_values(by=["file_name"])
df_sorted.head()

,audio_clipping,audio_clipping:confidence,background_noise_audible,background_noise_audible:confidence,overall_quality_of_the_audio,quiet_speaker,quiet_speaker:confidence,speaker_id,file_download,file_name,phrase,prompt,writer_id
6609,no_clipping,1.0000,light_noise,1.0,3.25,audible_speaker,1.0,13842059,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_13842059_104469105.wav,I have a painful cramp in my feet,Foot ache,44340112
2546,no_clipping,1.0000,light_noise,1.0,4.00,audible_speaker,1.0,13842059,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_13842059_105045085.wav,The pain feels like it's right below the skin,Muscle pain,44164300
551,no_clipping,0.6739,light_noise,1.0,3.33,audible_speaker,1.0,13842059,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_13842059_11964685.wav,I feel suicidal.,Emotional pain,44292353
1503,no_clipping,1.0000,light_noise,1.0,3.67,audible_speaker,1.0,13842059,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_13842059_12420758.wav,I feel a sharp pain in my ankle joint when I s...,Joint pain,38202325
2547,no_clipping,1.0000,light_noise,1.0,4.00,audible_speaker,1.0,13842059,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_13842059_13041979.wav,My shoulder hurts me so much,Shoulder pain,43985758


In [ ]:
df_sorted["audio"] = "/content/drive/MyDrive/Medical Speech, Transcription, and Intent/recordings/" + df_sorted["file_name"]

In [ ]:
df_sorted.head()

,audio_clipping,audio_clipping:confidence,background_noise_audible,background_noise_audible:confidence,overall_quality_of_the_audio,quiet_speaker,quiet_speaker:confidence,speaker_id,file_download,file_name,phrase,prompt,writer_id,audio
6609,no_clipping,1.0000,light_noise,1.0,3.25,audible_speaker,1.0,13842059,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_13842059_104469105.wav,I have a painful cramp in my feet,Foot ache,44340112,"/content/drive/MyDrive/Medical Speech, Transcr..."
2546,no_clipping,1.0000,light_noise,1.0,4.00,audible_speaker,1.0,13842059,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_13842059_105045085.wav,The pain feels like it's right below the skin,Muscle pain,44164300,"/content/drive/MyDrive/Medical Speech, Transcr..."
551,no_clipping,0.6739,light_noise,1.0,3.33,audible_speaker,1.0,13842059,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_13842059_11964685.wav,I feel suicidal.,Emotional pain,44292353,"/content/drive/MyDrive/Medical Speech, Transcr..."
1503,no_clipping,1.0000,light_noise,1.0,3.67,audible_speaker,1.0,13842059,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_13842059_12420758.wav,I feel a sharp pain in my ankle joint when I s...,Joint pain,38202325,"/content/drive/MyDrive/Medical Speech, Transcr..."
2547,no_clipping,1.0000,light_noise,1.0,4.00,audible_speaker,1.0,13842059,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_13842059_13041979.wav,My shoulder hurts me so much,Shoulder pain,43985758,"/content/drive/MyDrive/Medical Speech, Transcr..."


In [ ]:
train = df_sorted.iloc[0:5661, :]
test = df_sorted.iloc[5661:, :]

print(train.shape, test.shape)

(5661, 14) (1000, 14)


In [ ]:
train.to_csv("/content/drive/MyDrive/Medical Speech, Transcription, and Intent/train.csv", index=False)
test.to_csv("/content/drive/MyDrive/Medical Speech, Transcription, and Intent/test.csv", index=False)

In [ ]:
from datasets import Audio, Value, Features, load_dataset

In [ ]:
features = Features(
    {
        "phrase": Value("string"),
        "file_name": Value("string"),
        "audio": Audio(sampling_rate=16000)
    }
)

In [ ]:
sample_data = load_dataset(
    'csv', data_files={
        'train': "/content/drive/MyDrive/Medical Speech, Transcription, and Intent/train.csv",
        'test': "/content/drive/MyDrive/Medical Speech, Transcription, and Intent/test.csv"
    }
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
sample_data

DatasetDict({
    train: Dataset({
        features: ['audio_clipping', 'audio_clipping:confidence', 'background_noise_audible', 'background_noise_audible:confidence', 'overall_quality_of_the_audio', 'quiet_speaker', 'quiet_speaker:confidence', 'speaker_id', 'file_download', 'file_name', 'phrase', 'prompt', 'writer_id', 'audio'],
        num_rows: 5661
    })
    test: Dataset({
        features: ['audio_clipping', 'audio_clipping:confidence', 'background_noise_audible', 'background_noise_audible:confidence', 'overall_quality_of_the_audio', 'quiet_speaker', 'quiet_speaker:confidence', 'speaker_id', 'file_download', 'file_name', 'phrase', 'prompt', 'writer_id', 'audio'],
        num_rows: 1000
    })
})

In [ ]:
data = {
    "audio": ["/content/drive/MyDrive/Medical Speech, Transcription, and Intent/recordings/" + filename for filename in df_sorted["file_name"]],
    "sentence": df_sorted["phrase"].tolist(),
}
dataset = Dataset.from_dict(data)
dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 6661
})

In [ ]:
sample_data = sample_data.remove_columns(['audio_clipping', 'audio_clipping:confidence', 'background_noise_audible', 'background_noise_audible:confidence', 'overall_quality_of_the_audio', 'quiet_speaker', 'quiet_speaker:confidence', 'speaker_id', 'file_download', 'prompt', 'writer_id'])

In [ ]:
sample_data

DatasetDict({
    train: Dataset({
        features: ['file_name', 'phrase', 'audio'],
        num_rows: 5661
    })
    test: Dataset({
        features: ['file_name', 'phrase', 'audio'],
        num_rows: 1000
    })
})

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [ ]:
sample_data = sample_data.cast(features)

Casting the dataset:   0%|          | 0/5661 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
sample_data["train"][0]

{'phrase': 'I have a painful cramp in my feet',
 'file_name': '1249120_13842059_104469105.wav',
 'audio': {'path': '/content/drive/MyDrive/Medical Speech, Transcription, and Intent/recordings/1249120_13842059_104469105.wav',
  'array': array([-0.00383596, -0.00435373, -0.00818443, ..., -0.01057927,
         -0.02166483, -0.02287281]),
  'sampling_rate': 16000}}

In [ ]:
def prepare_dataset_sample(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["phrase"]).input_ids
    return batch

In [ ]:
sample_data = sample_data.map(prepare_dataset_sample, remove_columns=sample_data.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/5661 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
import pickle
file_path = '/content/drive/MyDrive/Medical Speech, Transcription, and Intent/medicaldata.pkl'

# Open the file in binary mode
with open(file_path, 'wb') as file:
    # Use pickle.dump() to save the data/object to the file
    pickle.dump(sample_data, file)

In [ ]:
with open(file_path, 'rb') as f:
  sample_data_loaded = pickle.load(f)

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
import transformers

print(transformers.__version__)


4.28.0


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-en-medical",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=600,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=50,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    remove_unused_columns=False
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=sample_data_loaded["train"],
    eval_dataset=sample_data_loaded["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,

)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/YashNikam10/whisper-small-en-medical into local empty directory.


In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
100,0.716600,0.636309,32.252671
200,0.196600,0.215488,12.568509
300,0.122600,0.160930,9.484440
400,0.056900,0.125614,8.202508
500,0.060000,0.114932,8.100325
600,0.055800,0.110132,7.607989


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=600, training_loss=0.4408154473702113, metrics={'train_runtime': 7094.8847, 'train_samples_per_second': 1.353, 'train_steps_per_second': 0.085, 'total_flos': 2.76955408687104e+18, 'train_loss': 0.4408154473702113, 'epoch': 1.69})

In [ ]:
kwargs = {
    "dataset_tags": "medical_speech_transcription",
    "dataset": "Medical Speech, Transcription, and Intent",
    "dataset_args": "config: en, split: test",
    "language": "en",
    "model_name": "Whisper Small Model Finetuned on Medical Speech Transcription",
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.push_to_hub(**kwargs)